In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import glob
import shutil
import torchvision

from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes
from shutil import copyfile
from pathlib import Path
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from PIL import Image, ImageDraw, ImageFont
from PIL import __version__ as pil_version
#from ultralytics.yolo.utils import LOGGER, TryExcept, plt_settings, threaded
#from ultralytics.yolo.utils.plotting import *
from scipy.ndimage import gaussian_filter1d

%matplotlib inline

In [ ]:
TRAIN_PATH = '/kaggle/input/global-wheat-detection/train/'
TEST_PATH = '/kaggle/input/global-wheat-detection/test/'

%cd ../

df = pd.read_csv('/kaggle/input/global-wheat-detection/train.csv')
df['path'] = df.apply(lambda row: TRAIN_PATH+row.image_id+'.jpg', axis=1)
df.head()

In [4]:
train_list, valid_list = train_test_split(df.image_id.unique(), test_size=0.2, random_state=1,shuffle = True)

In [ ]:
len(train_list)

In [ ]:
len(valid_list)

In [ ]:
train_df = df.loc[df.image_id.isin(train_list)]
valid_df = df.loc[df.image_id.isin(valid_list)]

train_df.loc[:, 'split'] = 'train'
valid_df.loc[:, 'split'] = 'valid'

df = pd.concat([train_df, valid_df]).reset_index(drop=True)

In [ ]:
len(train_df), len(valid_df)

In [ ]:
print(f'Size of dataset: {len(df)}, training images: {len(train_df)}. validation images: {len(valid_df)}')

In [ ]:
%cd ../../
%cd kaggle/working/

os.makedirs('dataset/images/train', exist_ok=True)
os.makedirs('dataset/images/valid', exist_ok=True)
os.makedirs('dataset/images/test', exist_ok=True)

os.makedirs('dataset/labels/train', exist_ok=True)
os.makedirs('dataset/labels/valid', exist_ok=True)
os.makedirs('dataset/labels/test', exist_ok=True)
os.makedirs('dataset/checkpoint', exist_ok=True)


%cd ../../
!tree kaggle/working/dataset

In [ ]:
# Move the images to relevant split folder.
for i in tqdm(range(len(df))):
    row = df.loc[i]
    if row.split == 'train':
        copyfile(row.path, f'kaggle/working/dataset/images/train/{row.image_id}.jpg')
    else:
        copyfile(row.path, f'kaggle/working/dataset/images/valid/{row.image_id}.jpg')

In [12]:
# Move the test images to relevant split folder.
# paths
src = 'kaggle/input/global-wheat-detection/test'
trg = 'kaggle/working/dataset/images/test'

files=os.listdir(src)

# iterating over all the files in
# the source directory
for fname in files:
    # copying the files to the
    # destination directory
    shutil.copy2(os.path.join(src,fname), trg)

In [ ]:
%cd /kaggle/working/

In [ ]:
%%writefile dataset.yaml
path: ../../../../../kaggle/working/dataset
train: images/train
val: images/valid
test: images/test

# Classes
names:
    0: wheat

In [ ]:
%cat dataset.yaml

In [16]:
def get_bbox(row):
    bboxes = []
    bbox = []
    list1 = row.bbox.replace("[", "").replace("]","").replace(",","")
    for i in list1.split(" "):
        bboxes.append(float(i))
    return bboxes

In [17]:
# Convert the bounding boxes in YOLO format.
def get_yolo_format_bbox(img_w, img_h, bboxes):
    yolo_boxes = []
    x = bboxes[0]
    y = bboxes[1]
    w = bboxes[2] 
    h = bboxes[3] 
    xc = x + int(w/2) # xmin + width/2
    yc = y + int(h/2) # ymin + height/2
    yolo_boxes=[0, xc/img_w, yc/img_h, w/img_w, h/img_h] # x_center y_center width height
    yolo_boxes = str(yolo_boxes).replace("[", "").replace("]","").replace(",","")
    return yolo_boxes

In [ ]:
# Prepare the txt files for bounding box
for i in tqdm(range(len(df))):
    row = df.loc[i]
    # Get image id
    img_id = row.image_id
    # Get split
    split = row.split
    
    if row.split=='train':
        file_name = f'dataset/labels/train/{row.image_id}.txt'
    else:
        file_name = f'dataset/labels/valid/{row.image_id}.txt'
        
    bboxes = get_bbox(row)
        
    # Format for YOLOv5
    IMG_SIZE = 1024
    yolo_bboxes = get_yolo_format_bbox(IMG_SIZE, IMG_SIZE, bboxes)
        
    with open(file_name, 'a') as f:
        f.write(yolo_bboxes)
        f.write('\n')

In [19]:
%%capture
#!pip install super-gradients==3.1.0
#!pip install super-gradients
!pip install imutils

In [ ]:
#Install YOLO-NAS
!pip install super-gradients==3.1.1 --quiet
!pip install --upgrade pillow --quiet
!pip install --upgrade torchvision --quiet
#vis utils
!pip install supervision --quiet 

In [ ]:
import os

import requests
import torch
from PIL import Image

from super_gradients.training import Trainer, dataloaders, models
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train, coco_detection_yolo_format_val
)
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import (
    PPYoloEPostPredictionCallback
)

In [22]:
class config:
    #trainer params
    CHECKPOINT_DIR = '/kaggle/working/dataset/checkpoint' #specify the path you want to save checkpoints to
    EXPERIMENT_NAME = 'wheat_YOLO-NAS' #specify the experiment name

    #dataset params
    DATA_DIR = '/kaggle/working/dataset' #parent directory to where data lives

    TRAIN_IMAGES_DIR = 'images/train' #child dir of DATA_DIR where train images are
    TRAIN_LABELS_DIR = 'labels/train' #child dir of DATA_DIR where train labels are

    VAL_IMAGES_DIR = 'images/valid' #child dir of DATA_DIR where validation images are
    VAL_LABELS_DIR = 'labels/valid' #child dir of DATA_DIR where validation labels are

    # if you have a test set
    TEST_IMAGES_DIR = 'images/test' #child dir of DATA_DIR where test images are
    TEST_LABELS_DIR = 'labels/test' #child dir of DATA_DIR where test labels are

    CLASSES = ['wheat'] #what class names do you have

    NUM_CLASSES = len(CLASSES)

    #dataloader params - you can add whatever PyTorch dataloader params you have
    #could be different across train, val, and test
    DATALOADER_PARAMS={
    'batch_size':8,
    'num_workers':2
    }

    # model params
    MODEL_NAME = 'yolo_nas_m' # choose from yolo_nas_s, yolo_nas_m, yolo_nas_l
    PRETRAINED_WEIGHTS = 'coco' #only one option here: coco
    

In [23]:
trainer = Trainer(experiment_name=config.EXPERIMENT_NAME, ckpt_root_dir=config.CHECKPOINT_DIR)

In [ ]:
train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': config.DATA_DIR,
        'images_dir': config.TRAIN_IMAGES_DIR,
        'labels_dir': config.TRAIN_LABELS_DIR,
        'classes': config.CLASSES
    },
    dataloader_params=config.DATALOADER_PARAMS
)


In [ ]:
val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': config.DATA_DIR,
        'images_dir': config.VAL_IMAGES_DIR,
        'labels_dir': config.VAL_LABELS_DIR,
        'classes': config.CLASSES
    },
    dataloader_params=config.DATALOADER_PARAMS
)


In [26]:


#test_data = coco_detection_yolo_format_val(
    #dataset_params={
      #  'data_dir': config.DATA_DIR,
     #   'images_dir': config.TEST_IMAGES_DIR,
    #    'labels_dir': config.TEST_LABELS_DIR,
   #     'classes': config.CLASSES
  #  },
 #   dataloader_params=config.DATALOADER_PARAMS
#)

In [ ]:
train_data.dataset.plot()

In [ ]:
model = models.get(config.MODEL_NAME, 
                   num_classes=config.NUM_CLASSES, 
                   pretrained_weights=config.PRETRAINED_WEIGHTS
                   )

In [29]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback 

train_params = {
    # ENABLING SILENT MODE
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 6,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    # ONLY TRAINING FOR 10 EPOCHS FOR THIS EXAMPLE NOTEBOOK
    "max_epochs": 100,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        # NOTE: num_classes needs to be defined here
        num_classes=config.NUM_CLASSES,
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            # NOTE: num_classes needs to be defined here
            num_cls=config.NUM_CLASSES,
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

In [ ]:
trainer.train(model=model, 
              training_params=train_params, 
              train_loader=train_data, 
              valid_loader=val_data)


In [31]:
best_model = models.get(config.MODEL_NAME,
                        num_classes=config.NUM_CLASSES,
                        checkpoint_path=os.path.join(config.CHECKPOINT_DIR, config.EXPERIMENT_NAME, 'average_model.pth'))

In [ ]:
best_model.predict( "/kaggle/working/dataset/images/test",  conf=0.25).show()

In [33]:
# best_model = models.get(
#     MODEL_ARCH,
#     num_classes=len(dataset_params['classes']),
#     checkpoint_path=f"{CHECKPOINT_DIR}/{EXPERIMENT_NAME}/average_model.pth"
# ).to(DEVICE)

In [34]:
# #trainer.test(
#  #   model=best_model,
#     test_loader=test_data,
#     test_metrics_list=DetectionMetrics_050(
#         score_thres=0.1, 
#         top_k_predictions=300, 
#         num_cls=config.NUM_CLASSES, 
#         normalize_targets=True, 
#         post_prediction_callback=PPYoloEPostPredictionCallback(
#             score_threshold=0.01, 
#             nms_top_k=1000, 
#             max_predictions=300,                                                                              
#             nms_threshold=0.7
#         )
#     )
# )

In [35]:
# #trainer.test(model=best_model,
#  #           test_loader=test_loader,
#   #          test_metrics_list=DetectionMetrics_050(score_thres=0.1, 
#    #                                                top_k_predictions=300, 
#     #                                               num_cls=config.NUM_CLASSES, 
#      #                                              normalize_targets=True, 
#       #                                             post_prediction_callback=PPYoloEPostPredictionCallback(score_threshold=0.01, 
#                                                                                                           nms_top_k=1000,                                                                                                           max_predictions=300,                                                                              nms_threshold=0.7)
#        #                                           ))

In [36]:
#%matplotlib inline
#plot_results('/kaggle/working/runs/detect/train/results.csv')

In [37]:
#def show_results(file):
 #   plt.figure(figsize=(30,30))
  #  img = np.asarray(Image.open(file))
   # plt.imshow(img)
   # plt.axis('off');

In [38]:
#show_results('/kaggle/working/runs/detect/train/results.png')

In [39]:
#copyfile('/kaggle/working/runs/detect/train/results.png','/kaggle/working/results.png')
#copyfile('/kaggle/working/runs/detect/train/results.csv','/kaggle/working/results.csv')
#copyfile('/kaggle/working/runs/detect/train/weights/best.pt','/kaggle/working/best.pt')